# Bayesian Search Algorithm

Use bayesian search based on the paper: "Bayesian-Based Search Decision Framework and Search Strategy Analysis in Probabilistic Search"

link: https://onlinelibrary.wiley.com/doi/full/10.1155/2020/8865381

In [3]:
# Import necessary libraries
import networkx as nx
import random
import xml.etree.ElementTree as ET

In [4]:
# Constants
map_graph_path = "../graphs/outputs/undirected_graph.graphml" # .graphml file path
start_node = "n0"   # Start position (node)
target_node = "n20" # target's (ring) position (node)
p_tp = 0.8  # P(true positive)
p_fp = 0.2  # P(false positive)
p_tn = 0.9 # P(true negative)
p_fn = 0.1  # P(false negative)
B_lower = 0  # Lower boundary for maximum belief
B_upper = 1   # Upper boundary for maximum belief
max_steps = 500  # Maximum number of steps

# May I say "Fuck this paper's notations. What the fuck do you mean by 3 same equations having 3 distinct values??? What the hell is this level of inconsistency in naming variables???."

In [5]:
# Generic helper functions

# Display graph information
def display_graph_information(graph):
    # Print nodes data
    for node, data in graph.nodes(data=True):
        print(f"Node: {node}, Attributes: {data}")

    # Print edges data
    for u, v, data in graph.edges(data=True):
        print(f"Edge: ({u}, {v}), Attributes: {data}")

    print("Number of Nodes:", len(graph.nodes()))
    print("Number of Edges:", len(graph.edges()))

def add_edges_weight(graph):
    for u, v, data in graph.edges(data=True):
        data['weight'] = 1.0  # Assign a weight of 1 to each edge

In [6]:
# Set target (ring) position (node)
def set_target_node(graph, target_node_id):
    nx.set_node_attributes(graph, {target_node_id: {"target": True}})

In [7]:
# Bayesian search for incomplete information sensor

# Create an initial belief map
def initialize_belief_map(graph, rand=False):
    num_nodes = len(graph.nodes())

    if rand:
        for node in graph.nodes():
            graph.nodes[node]["P(target)"] = random.uniform(0.0, 1.0)
    else:
        uniform_belief = 1.0 / num_nodes

        for node in graph.nodes():
            graph.nodes[node]["P(target)"] = uniform_belief


# Implement the Dijkstra algorithm
def dijkstra_search(graph, start_node, target_node):
    try:
        path = nx.dijkstra_path(graph, source=start_node, target=target_node, weight='weight')
        
        return path
    except nx.NetworkXNoPath:
        print(f"No path found between {start_node} and {target_node}.")
        return []

# Update the belief map
def update_belief_map(graph, agent_node, observation):
    total_nodes = len(graph.nodes())
    new_beliefs = {}
    
    # Calculate the denominator for the Bayesian update (marginal probability)
    denominator = 0.0

    for node in graph.nodes():
        prior_prob = graph.nodes[node]["P(target)"]

        if node == agent_node:
            if observation:
                denominator += prior_prob * p_tp  # P(O | D_d) * P(D_d)
            else:
                denominator += prior_prob * p_fn  # P(~O | D_d) * P(D_d)
        else:
            if observation:
                denominator += prior_prob * p_fp  # P(O | D_c) * P(D_c)
            else:
                denominator += prior_prob * p_tn  # P(~O | D_c) * P(D_c)

    # Quit if denominator = 0
    if denominator == 0:
        return

    # Updated probabilities
    new_beliefs = {}

    for node in graph.nodes():
        prior_prob = graph.nodes[node]["P(target)"]

        if node == agent_node:
            if observation:
                numerator = prior_prob * p_tp  # P(D_d | O)
            else:
                numerator = prior_prob * p_fn  # P(D_d | ~O)
        else:
            if observation:
                numerator = prior_prob * p_fp  # P(D_c | O)
            else:
                numerator = prior_prob * p_tn  # P(D_c | ~O)
                
        updated_prob = numerator / denominator
        new_beliefs[node] = updated_prob

    # Assign the new beliefs to the graph nodes
    nx.set_node_attributes(graph, new_beliefs, "P(target)")

# Modified saccadic search strategy
def saccadic_search_strategy(graph, start_node, max_steps, target_node):
    current_node = start_node
    path_taken = [current_node]
    target_found = False
    
    # Initialize with uniform belief
    initialize_belief_map(graph, rand=False)

    current_path = []
    
    for step in range(max_steps):
        # print(f"Step: {step + 1}")
        
        # Check if we have reached our planned destination and need a new one
        if not current_path:
            # Find the node with the highest belief
            highest_belief_node = max(graph.nodes(data='P(target)'), key=lambda x: x[1])[0]
            
            # print(f"New target destination identified: {highest_belief_node}.")
            
            # Plan a new path to the highest belief node
            current_path = dijkstra_search(graph, current_node, highest_belief_node)
            
            # If no path is found, break the loop
            if not current_path:
                # print(f"No path found to {highest_belief_node}.")
                
                break
                
            # The first element is the current node, we want to move to the next.
            if len(current_path) > 1:
                current_path.pop(0)

        # Move to the next node on the path
        if current_path:
            next_node = current_path.pop(0)

            # print(f"Moving from {current_node} to {next_node}")

            current_node = next_node

            path_taken.append(current_node)
        
        # Perform observation and update belief at the new location
        is_target = graph.nodes[current_node].get("target", False)
        
        if is_target:
            observation = random.random() < p_tp   # True positive with probability p_tp
        else:
            observation = random.random() < p_fp   # False positive with probability p_fp
        
        update_belief_map(graph, current_node, observation)
        # print(f"At {current_node}. Belief at this node is {graph.nodes[current_node]['P(target)']:.4f}")

        # Check for target presence. We check at every step because we're moving and observing.
        if current_node == target_node:
            #  print(f"Target found at {current_node}!")

             target_found = True

             break
    
    return path_taken, target_found

In [986]:
# Read .graphml file
map_graph = nx.read_graphml(map_graph_path)

display_graph_information(map_graph)

Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101'}
Node: n1, Attributes: {'x': '491.39296206082173', 'y': '814.701509196422'}
Node: n2, Attributes: {'x': '540.242019608562', 'y': '765.0825618080634'}
Node: n3, Attributes: {'x': '589.4380120008074', 'y': '813.8610105825668'}
Node: n4, Attributes: {'x': '491.50793350674974', 'y': '764.5725599826202'}
Node: n5, Attributes: {'x': '539.5620171746378', 'y': '862.7626250900943'}
Node: n6, Attributes: {'x': '490.88296023537856', 'y': '862.2526232646511'}
Node: n7, Attributes: {'x': '590.2080004396669', 'y': '764.5725599826202'}
Node: n8, Attributes: {'x': '589.0179961802995', 'y': '863.4426275240185'}
Node: n9, Attributes: {'x': '539.3920165661567', 'y': '716.6725173889456'}
Node: n10, Attributes: {'x': '589.6080126092883', 'y': '716.3325161719835'}
Node: n11, Attributes: {'x': '492.18793594067404', 'y': '718.2025228652752'}
Node: n12, Attributes: {'x': '443.9136948765907', 'y': '718.5026128475819'}
Node: n13, Attribute

In [987]:
# Run the search and get results
print("\nStarting Saccadic Search...")

search_path, found = saccadic_search_strategy(map_graph, start_node, max_steps, target_node=target_node)

print("\nSearch Results:")  

if found:
    print("Target was found!")
else:
    print("Target was not found within the maximum number of steps.")
    
print("Search path:", search_path)


Starting Saccadic Search...

Search Results:
Target was found!
Search path: ['n0', 'n0', 'n0', 'n1', 'n2', 'n3', 'n0', 'n4', 'n1', 'n5', 'n6', 'n6', 'n0', 'n7', 'n3', 'n8', 'n8', 'n3', 'n7', 'n9', 'n9', 'n10', 'n10', 'n9', 'n11', 'n11', 'n11', 'n11', 'n12', 'n14', 'n13', 'n6', 'n15', 'n8', 'n16', 'n3', 'n0', 'n1', 'n17', 'n17', 'n6', 'n5', 'n3', 'n18', 'n16', 'n19', 'n16', 'n18', 'n20']


In [988]:
for searched_node in search_path:
    for node, data in map_graph.nodes(data=True):
        if searched_node == node:
            print(f"Node: {node}, Attributes: {data}")
            print("X:", int(float(data["x"]) / 50))
            print("Y:", int(float(data["y"]) / 50))

Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101', 'P(target)': 3.115549829397224e-05}
X: 10
Y: 16
Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101', 'P(target)': 3.115549829397224e-05}
X: 10
Y: 16
Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101', 'P(target)': 3.115549829397224e-05}
X: 10
Y: 16
Node: n1, Attributes: {'x': '491.39296206082173', 'y': '814.701509196422', 'P(target)': 1.9472186433732647e-06}
X: 9
Y: 16
Node: n2, Attributes: {'x': '540.242019608562', 'y': '765.0825618080634', 'P(target)': 0.0001577247101132344}
X: 10
Y: 15
Node: n3, Attributes: {'x': '589.4380120008074', 'y': '813.8610105825668', 'P(target)': 2.4039736337941543e-08}
X: 11
Y: 16
Node: n0, Attributes: {'x': '539.7320177831189', 'y': '814.3710124080101', 'P(target)': 3.115549829397224e-05}
X: 10
Y: 16
Node: n4, Attributes: {'x': '491.50793350674974', 'y': '764.5725599826202', 'P(target)': 0.0001577247101132344}
X: 9
Y: 15
Node: n1, Attrib

# Utils

In [18]:
import re
import xml.etree.ElementTree as ET

LABEL_RE = re.compile(r"^\s*-?\d+\s*,\s*-?\d+\s*$")
NS = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml",
}
map_graph_path = "../mapGraphs/graphml/HIMCM_graph_FINAL.graphml" # .graphml file path
def inject_node_labels_from_graphml_file(G, graphml_path, label_attr="node_label"):
    tree = ET.parse(str(graphml_path))
    root = tree.getroot()

    for n in root.findall(".//g:graph/g:node", NS):
        nid = n.get("id")
        if nid not in G:  # only inject for nodes that exist in this graph
            continue

        txt = None
        for d in n.findall("./g:data", NS):
            lab = d.find("./y:ShapeNode/y:NodeLabel", NS)
            if lab is None:
                lab = d.find("./y:GenericNode/y:NodeLabel", NS)
            if lab is not None and lab.text:
                txt = lab.text.strip()
                break

        # If it's a coordinate-like label, use it; otherwise keep any existing value
        if txt and LABEL_RE.match(txt):
            G.nodes[nid][label_attr] = txt

def nodes_to_labels(G, node_id_path, label_attr="node_label"):
    return [G.nodes[nid].get(label_attr, str(nid)) for nid in node_id_path]


## Test Cases

### Static Test

In [20]:
start_nodes = ["n0", "n20", "n33", "n200", "n555"]
target_nodes = ["n20", "n22", "n576", "n714", "n123"]
max_steps = 15000

In [21]:
def static_test(graph, start_nodes, target_nodes):
    for start_node, target_node in zip(start_nodes, target_nodes):
        print("Start Node: " + start_node, "Target Node: " + target_node)

        search_path, found = saccadic_search_strategy(graph, start_node, max_steps, target_node)
        inject_node_labels_from_graphml_file(graph, map_graph_path, label_attr="y:LabelModel")
        # print("\nSearch Results:")  

        # if found:
        #     print("Target was found!")
        # else:
        #     print("Target was not found within the maximum number of steps.")
        
        path_labels = nodes_to_labels(graph, search_path, label_attr="y:LabelModel")
        print("Path (labels):", " -> ".join(path_labels))
        print("Target found:", found)


In [22]:
map_graph = nx.read_graphml(map_graph_path)

static_test(map_graph, start_nodes, target_nodes)


Start Node: n0 Target Node: n20
Path (labels): 16,11 -> 16,11 -> 16,10 -> 15,11 -> 16,12 -> 16,12 -> 16,11 -> 15,10 -> 16,10 -> 17,11 -> 17,10 -> 17,10 -> 17,10 -> 17,10 -> 16,11 -> 15,12 -> 16,12 -> 17,12 -> 17,12 -> 16,11 -> 15,11 -> 14,11 -> 14,12 -> 15,11 -> 14,10 -> 14,10 -> 14,9 -> 15,9 -> 16,9 -> 16,9 -> 16,9 -> 16,9 -> 16,9 -> 17,10 -> 18,11 -> 17,12 -> 16,13 -> 16,12 -> 16,11 -> 17,10 -> 17,9 -> 17,10 -> 16,11 -> 15,11 -> 14,12 -> 15,13 -> 16,13 -> 17,13 -> 16,13 -> 15,14 -> 14,13
Target found: True
Start Node: n20 Target Node: n22
Path (labels): 14,13 -> 14,12 -> 15,11 -> 16,11 -> 16,10 -> 16,11 -> 16,12 -> 16,12 -> 16,11 -> 15,10 -> 16,10 -> 17,11 -> 17,10 -> 16,11 -> 15,12 -> 16,12 -> 17,12 -> 16,11 -> 15,11 -> 14,11 -> 14,10 -> 14,9 -> 15,9 -> 16,9 -> 17,10 -> 18,11 -> 17,12 -> 16,13 -> 16,12 -> 16,11 -> 17,10 -> 17,9 -> 17,10 -> 16,11 -> 15,11 -> 14,12 -> 15,13 -> 15,13 -> 14,12 -> 15,11 -> 16,12 -> 17,13 -> 17,13 -> 16,13 -> 15,14 -> 14,13 -> 14,12 -> 15,11 -> 16,11 -> 1

### Random Test

In [998]:
number_of_test_cases = 5
max_steps = 15000


In [999]:
def random_test(graph, number_of_test_cases):
    number_of_nodes = len(graph.nodes())
    start_nodes = []
    target_nodes = []

    for i in range(number_of_test_cases):
        start_nodes.append("n" + str(random.randint(0, number_of_nodes)))
        target_nodes.append(("n" + str(random.randint(0, number_of_nodes))))
    
    static_test(graph, start_nodes, target_nodes)

In [1000]:
map_graph = nx.read_graphml(map_graph_path)

random_test(map_graph, number_of_test_cases)

Start Node: n103 Target Node: n649

Search Results:
Target was found!
Search path: ['n103', 'n108', 'n122', 'n10', 'n7', 'n0', 'n7', 'n10', 'n122', 'n108', 'n122', 'n10', 'n2', 'n1', 'n0', 'n3', 'n0', 'n4', 'n4', 'n4', 'n4', 'n4', 'n1', 'n5', 'n6', 'n5', 'n8', 'n3', 'n7', 'n9', 'n11', 'n12', 'n14', 'n13', 'n13', 'n6', 'n15', 'n8', 'n16', 'n3', 'n0', 'n1', 'n17', 'n6', 'n5', 'n3', 'n18', 'n16', 'n19', 'n16', 'n18', 'n20', 'n18', 'n3', 'n5', 'n21', 'n15', 'n22', 'n15', 'n21', 'n23', 'n21', 'n15', 'n22', 'n24', 'n22', 'n15', 'n21', 'n23', 'n49', 'n33', 'n25', 'n26', 'n27', 'n28', 'n25', 'n29', 'n26', 'n30', 'n31', 'n25', 'n32', 'n34', 'n35', 'n34', 'n36', 'n37', 'n39', 'n38', 'n31', 'n40', 'n33', 'n41', 'n28', 'n25', 'n26', 'n42', 'n42', 'n31', 'n30', 'n28', 'n43', 'n41', 'n44', 'n41', 'n43', 'n45', 'n45', 'n43', 'n28', 'n30', 'n46', 'n46', 'n30', 'n28', 'n43', 'n43', 'n43', 'n43', 'n43', 'n43', 'n43', 'n41', 'n44', 'n47', 'n40', 'n46', 'n48', 'n48', 'n46', 'n40', 'n47', 'n49', 'n23', 'n2